In [ ]:
from pathlib import Path
import json
import requests

root_dir = Path.cwd().parent



We are hacking access to the CRM API by using personal auth cookie information from the browser.

ENDPOINT = https://dfdscrm.crm4.dynamics.com/api/data/v9.2/leads?$top=10

1. Go to browser dev tools -> Network
2. Find any request to leads
3. In request's header, copy the 'Cookie' value
4. Paste it below
5. Generate cookies.py with new cookie
Done


In [ ]:
cookie_string = """[PASTE VALUE HERE]"""


Some parsing of the cookies
And saving to cookies.py files

In [ ]:
def parse_cookie_string(cookie_string):
    cookies = {}

    if not cookie_string:
        raise ValueError("Cookie string is empty")

    # Split by semicolon and parse each cookie
    for cookie in cookie_string.split(';'):
        cookie = cookie.strip()
        if '=' in cookie:
            name, value = cookie.split('=', 1)
            cookies[name.strip()] = value.strip()

    return cookies

def format_for_requests(cookies_dict):
    print("cookies = {")
    for name, value in cookies_dict.items():
        print(f"    '{name}': '{value}',")
    print("}")
    return cookies_dict

cookies = parse_cookie_string(cookie_string)
formatted_cookies = format_for_requests(cookies)

with open('cookies.py', 'w') as f:
    f.write("cookies = {\n")
    for name, value in cookies.items():
        f.write(f"    '{name}': '{value}',\n")
    f.write("}\n")


# Get Leads from CRM

These also include associations (such as company, collection, delivery countries)

In [ ]:
from models.lead import Lead


def get_crm_leads_url_expanded() -> str:
    # select = "leadid,fullname"
    select = None
    expand_props = [
        "dfds_collectioncountry($select=dfds_alpha2code,dfds_alpha3code,dfds_name)",
        "dfds_deliverycountry($select=dfds_alpha2code,dfds_alpha3code,dfds_name)",
        "dfds_countrylookup($select=dfds_alpha2code,dfds_alpha3code,dfds_name)"
    ]

    url = "https://dfdscrm.crm4.dynamics.com/api/data/v9.2/leads"

    if select:
        url += f"?$select={select}"

    if expand_props:
        url += f"?$expand={",".join(expand_props)}"

    return url


def get_crm_leads(sample_size: int = 50) -> requests.Session:
    headers = {
        'Content-Type': 'application/json',
        'OData-MaxVersion': '4.0',
        'OData-Version': '4.0',
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36'
    }

    # Create session with cookies
    session = requests.Session()
    session.cookies.update(cookies)

    url = get_crm_leads_url_expanded()
    final_url = f"{url}&$top={sample_size}" if sample_size else url

    print(final_url)

    return session.get(
        final_url,
        headers=headers
    )

response = get_crm_leads(sample_size=1000)
response.status_code


In [ ]:
# Extract the actual leads

crm_leads = response.json()['value']
len(crm_leads)


In [ ]:
# Extract all available CRM lead columns


import pandas as pd

def extract_all_crm_lead_columns(crm_leads: list) -> list:
    columns = set()
    for lead in crm_leads:
        columns.update(lead.keys())

    return columns

crm_leads_columns = extract_all_crm_lead_columns(crm_leads)


In [ ]:
# And save leads to /data/crm_leads_full.csv

crm_df = pd.DataFrame(crm_leads, columns=list(crm_leads_columns))
crm_df.to_csv(root_dir / "data" / "crm_leads_full.csv", index=False)
